In [1]:
%load_ext cython

In [2]:
import itertools
import numpy as np
import time

In [5]:
actions = []
sequence = []
with open("./puzzle_inputs/22.txt") as f:
# with open("./22_sample.in") as f:
	for line in f:
		action, cuboid = line.strip().split()
		actions.append(int(action == "on"))
		sequence.append([[int(x) for x in v[2:].split('..')] for v in cuboid.split(',')])
		# [list(map(int(v[2:].split('..')))) for v in sequence.split(',')]
actions = np.array(actions, dtype=np.int32)
sequence = np.array(sequence, dtype=np.int32)
sequence[:,:,1] += 1

In [6]:
dims = sequence.shape[1]

In [7]:
%%cython

cdef unsigned long LONG_MASK = 0xFFFFFFFFFFFFFFFF

cpdef int has(unsigned long[:] bitset, int element):
	return bitset[element // 64] & (1l << (element % 64)) != 0

cpdef void add(unsigned long[:] bitset, int element):
	bitset[element // 64] |= (1l << (element % 64))

cpdef void remove(unsigned long[:] bitset, int element):
	bitset[element // 64] &= (LONG_MASK ^ (1l << (element % 64)))

cpdef int size(unsigned long[:] bitset):
	cdef int i
	cdef unsigned long v
	cdef total = 0
	for v in bitset:
		while v != 0:
			total += 1
			v = v & (v-1)
	return total

In [8]:
%%cython

cpdef int binary_search(int [:] arr, int elem):
	cdef int i, j, m
	i = 0
	j = len(arr) - 1
	while i <= j:
		m = (i+j)//2
		if arr[m] == elem:
			return m
		if elem < arr[m]:
			j = m - 1
		else:
			i = m + 1
	raise Exception("Not found")

cdef unsigned long LONG_MASK = 0xFFFFFFFFFFFFFFFF

cdef inline int has(unsigned long[:] bitset, int element):
	return bitset[element // 64] & (1l << (element % 64)) != 0

cdef inline void add(unsigned long[:] bitset, int element):
	bitset[element // 64] |= (1l << (element % 64))

cdef inline void remove(unsigned long[:] bitset, int element):
	bitset[element // 64] &= (LONG_MASK ^ (1l << (element % 64)))

cpdef unsigned long boot(int[:] vx, int[:] vy, int[:] vz, unsigned long[:,:,:] active_voxels, int[:] actions, int[:,:,:] sequence):
	cdef unsigned long total_active
	cdef int seq_id
	cdef int active
	cdef int x1, x2, y1, y2, z1, z2
	cdef int i, j, k
	cdef unsigned long dx, dy, dz, volume
	total_active = 0
	for seq_id in range(len(actions)):
		active = actions[seq_id]
		(x1, x2), (y1, y2), (z1, z2) = sequence[seq_id]
		print(f"\r{seq_id}/{len(sequence)}", end="")
		for i in range(binary_search(vx, x1), binary_search(vx, x2)):
			for j in range(binary_search(vy, y1), binary_search(vy, y2)):
				for k in range(binary_search(vz, z1), binary_search(vz, z2)):
					if active == has(active_voxels[i][j], k):
						continue

					# Compute the volume
					dx = vx[i+1] - vx[i]
					dy = vy[j+1] - vy[j]
					dz = vz[k+1] - vz[k]
					volume = dx*dy*dz

					if active:
						add(active_voxels[i][j], k)
						total_active += volume
					else:
						remove(active_voxels[i][j], k)
						total_active -= volume
	return total_active

In [9]:
voxels = []
voxel_lookup = []
for axis in range(dims):
	voxels.append(np.unique(sequence[:,axis,:]).astype(np.int32))
	voxel_lookup.append({ v:i for i,v in enumerate(voxels[-1]) })
voxels_shape = [len(v) for v in voxels]
voxels_shape

[832, 831, 829]

In [10]:
ints_per_dim = int(np.ceil(voxels_shape[-1] / 64))
ints_per_dim

13

In [11]:
active_voxels = np.zeros(tuple(v-1 for v in voxels_shape[:-1]) + (ints_per_dim,), dtype=np.uint64)

In [12]:
t = time.time()
solution = boot(*voxels, active_voxels, actions, sequence)
t = time.time() - t

print(f"Solution: {solution}; Time: {t:.2}s")

419/420Solution: 1263946820845866; Time: 5.5s


In [152]:
active_voxels.shape

(831, 830, 13)

In [87]:
total_active = 0
total = 0
active_voxels = np.zeros(tuple(v-1 for v in voxels_shape[:-1]) + (ints_per_dim,), dtype=np.uint64)
for seq_id, (active, coords) in enumerate(zip(actions, sequence)):
	print(f"\r{seq_id}/{len(sequence)}", end="")
	(x1, x2), (y1, y2), (z1, z2) = coords
	changed = 0
	for i in range(voxel_lookup[0][x1], voxel_lookup[0][x2]):
		for j in range(voxel_lookup[1][y1], voxel_lookup[1][y2]):
			for k in range(voxel_lookup[2][z1], voxel_lookup[2][z2]):
				# if active:
				# 	add(active_voxels[i,j], k)
				# else:
				# 	remove(active_voxels[i,j], k)

				if active == has(active_voxels[(i,j)], k):
					continue

				# Compute the volume
				dx = voxels[0][i+1] - voxels[0][i]
				dy = voxels[1][j+1] - voxels[1][j]
				dz = voxels[2][k+1] - voxels[2][k]
				volume = dx*dy*dz

				if active:
					add(active_voxels[(i,j)], k)
					total_active += volume
					changed += 1
					total += 1
				else:
					remove(active_voxels[(i,j)], k)
					total_active -= volume
					changed -= 1
					total -= 1
	# t = 0
	# for row in active_voxels:
	# 	for col in row:
	# 		t += size(col)
	# assert total == t
total_active

24/420

KeyboardInterrupt: 

In [107]:
def chamfer_distance(X, Y):
	distance = 0
	for x_row in X:
		min_d = 0xFFFFFFFF
		for y_row in Y:
			d = 0
			for i in range(len(x_row)):
				d += (y_row[i] - x_row[i])**2
			if d < min_d:
				min_d = d
		distance += min_d**(0.5)
	return distance / len(X)

In [125]:
def chamfer_distance_numpy(X, Y):
	distance = 0
	for x_row in X:
		min_d = 0xFFFFFFFF
		d = np.min(np.sum((b - a[0])**2, axis=1))
		if d < min_d:
			min_d = d
		distance += min_d**(0.5)
	return distance / len(X)

In [111]:
%%cython

cpdef double chamfer_distance_cython(double[:,:] a, double[:,:] b):
    cdef double total_dist = 0.0
    cdef double d
    cdef double min_d
    cdef int i, j, k
    cdef int len_a = len(a)
    cdef int len_b = len(b)
    cdef int point_width = len(a[0])
    for i in range(len_a):
        min_d = -1
        for j in range(len_b):
            d = 0.0
            for k in range(point_width):
                d += abs(b[j][k] - a[i][k])
            if d < min_d or min_d == -1.0:
                min_d = d
        total_dist += min_d
    return total_dist / len_a

In [119]:
a = np.random.uniform(size=(1000, 2))
b = np.random.uniform(size=(1000, 2))

In [124]:
np.sum((b - a[0])**2, axis=1)

array([4.86698473e-01, 1.02874479e+00, 9.13860528e-02, 1.35343260e-01,
       5.97606307e-01, 1.19196301e+00, 5.26004605e-01, 1.22194772e+00,
       5.29203022e-03, 1.08144401e-01, 5.13222538e-01, 3.84661631e-01,
       1.99168408e-03, 1.40583786e-01, 9.10926046e-01, 3.48242357e-01,
       4.27424434e-01, 1.23752615e+00, 2.30788130e-01, 2.43607270e-01,
       7.79772151e-01, 3.15798532e-01, 1.49624062e-01, 3.64297796e-01,
       8.75860897e-01, 4.43523588e-01, 3.59380141e-01, 4.64263259e-01,
       1.39691198e-01, 1.94213958e-01, 6.31342858e-01, 1.65500966e-01,
       2.87277900e-01, 8.94602697e-01, 5.87414853e-01, 6.18672737e-02,
       3.04765392e-01, 5.65313626e-02, 2.96243888e-01, 6.50309594e-01,
       1.87100334e-01, 1.04184493e-01, 1.49494079e-01, 5.50244222e-01,
       2.16293361e-01, 7.80929221e-01, 9.82514218e-01, 7.50351358e-01,
       1.39794475e+00, 2.08272173e-01, 7.33588820e-01, 8.40581864e-01,
       2.30540351e-01, 5.05712349e-01, 5.48732783e-01, 8.46167719e-01,
      

In [115]:
t = time.time()
chamfer_distance(a, b)
print(time.time() - t)

1.7479932308197021


In [126]:
chamfer_distance_numpy(a, b)

0.020144010858589485

In [116]:
t = time.time()
chamfer_distance_cython(a, b)
print(time.time() - t)

0.004709720611572266
